# Spotify - Search by Emotion Detected



Q1 as high valence high arousal (HVHA) - Happy,

Q2 as low valence high arousal (LVHA) - Angry,

Q3 as low valence low arousal (LVLA) - Sad,

Q4 as high valence low arousal (HVLA) - Calm.


In [ ]:
!pip install spotipy

In [ ]:
import json, requests
import numpy as np

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [ ]:
sp = spotipy.Spotify(
    auth_manager=SpotifyClientCredentials(
        client_id='EBMP_APP_CLIENT_ID',
        client_secret='EBMP_CLIENT_SECRET'
        )
    )

In [ ]:
all_genre_seeds = sp.recommendation_genre_seeds()

print(all_genre_seeds)

{'genres': ['acoustic', 'afrobeat', 'alt-rock', 'alternative', 'ambient', 'anime', 'black-metal', 'bluegrass', 'blues', 'bossanova', 'brazil', 'breakbeat', 'british', 'cantopop', 'chicago-house', 'children', 'chill', 'classical', 'club', 'comedy', 'country', 'dance', 'dancehall', 'death-metal', 'deep-house', 'detroit-techno', 'disco', 'disney', 'drum-and-bass', 'dub', 'dubstep', 'edm', 'electro', 'electronic', 'emo', 'folk', 'forro', 'french', 'funk', 'garage', 'german', 'gospel', 'goth', 'grindcore', 'groove', 'grunge', 'guitar', 'happy', 'hard-rock', 'hardcore', 'hardstyle', 'heavy-metal', 'hip-hop', 'holidays', 'honky-tonk', 'house', 'idm', 'indian', 'indie', 'indie-pop', 'industrial', 'iranian', 'j-dance', 'j-idol', 'j-pop', 'j-rock', 'jazz', 'k-pop', 'kids', 'latin', 'latino', 'malay', 'mandopop', 'metal', 'metal-misc', 'metalcore', 'minimal-techno', 'movies', 'mpb', 'new-age', 'new-release', 'opera', 'pagode', 'party', 'philippines-opm', 'piano', 'pop', 'pop-film', 'post-dubstep'

In [ ]:
class EmotionCalculator:
    @staticmethod
    def calculate_angle(arousal, valence):
        # Scale arousal and valence to [-1, 1] range
        arousal_scaled = 2 * arousal - 1
        valence_scaled = 2 * valence - 1
        # Calculate the angle in radians
        angle_rad = np.arctan2(arousal_scaled, valence_scaled)
        # Convert radians to degrees
        angle_deg = np.degrees(angle_rad)
        # Ensure angle is positive
        if angle_deg < 0:
            angle_deg += 360
        return angle_deg

    @staticmethod
    def determine_quadrant(angle_deg):
        if 0 <= angle_deg < 90:
            return "QUAD1"
        elif 90 <= angle_deg < 180:
            return "QUAD2"
        elif 180 <= angle_deg < 270:
            return "QUAD3"
        else:
            return "QUAD4"

    @staticmethod
    def determine_emotion(quadrant_name):
        if quadrant_name == "QUAD1":
            return "Happiness"
        elif quadrant_name == "QUAD2":
            return "Anger"
        elif quadrant_name == "QUAD3":
            return "Sadness"
        else:
            return "Calmness"

In [ ]:
# Calmness
calmness_genres = ['acoustic', 'ambient', 'classical', 'piano', 'relaxation']

# Anger
anger_genres = ['hardcore', 'heavy-metal', 'punk', 'rock', 'grindcore']

# Sadness
sadness_genres = ['blues', 'emotional', 'indie-folk', 'soul', 'sad']

# Happiness
happiness_genres = ['dance', 'disco', 'happy', 'pop', 'reggae']

In [ ]:
res_happy = sp.recommendations(
    seed_genres=happiness_genres,
    limit=30,
    country="US",
    min_energy=0.5,
    max_energy=1.0,
    target_energy=0.8,
    min_valence=0.5,
    max_valence=1.0,
    target_valence=0.8
)

res_sad = sp.recommendations(
    seed_genres=sadness_genres,
    limit=30,
    country="US",
    min_energy=0.1,
    max_energy=0.5,
    target_energy=0.3,
    min_valence=0.1,
    max_valence=0.5,
    target_valence=0.3
)

res_anger = sp.recommendations(
    seed_genres=anger_genres,
    limit=30,
    country="US",
    min_energy=0.5,
    max_energy=1.0,
    target_energy=0.8,
    min_valence=0.1,
    max_valence=0.5,
    target_valence=0.3
)

res_calm = sp.recommendations(
    seed_genres=calmness_genres,
    limit=30,
    country="US",
    min_energy=0.1,
    max_energy=0.5,
    target_energy=0.3,
    min_valence=0.5,
    max_valence=1.0,
    target_valence=0.8
)

In [ ]:
available_markets = ["US", "GB", "DE", "FR", "BR", "JP", "RU", "IN", "KR", "ES"]

def get_preview_url(track_id):
    for market in available_markets:
        try:
            output = sp.track(track_id, market=market)
            if output['preview_url']:
                return output['preview_url']
        except Exception as e:
            print(f"Error occurred: {e}")
    return "No preview url found"

In [ ]:
# Number of limits specified
limit = 30

# List to store track information
track_info_list = []

for i in range(min(limit, len(res_happy["tracks"]))):
    track_info = {}
    track = res_happy["tracks"][i]

    # Extracting track ID and name
    track_info['id'] = track['id']
    track_info['name'] = track['name']

    # Extracting artists' names
    track_info['artists'] = [artist['name'] for artist in track['artists']]

    # Extracting images details
    track_info['images'] = track['album']['images']

    # Extracting preview URL if available
    if 'preview_url' in track and track['preview_url'] is not None:
      track_info['preview_url'] = track['preview_url']
    else:
      track_info['preview_url'] = get_preview_url(track['id'])

    # Extract the track features
    track_features = sp.audio_features(track['id'])
    track_info['arousal'] = track_features[0]['energy']
    track_info['danceability'] = track_features[0]['danceability']
    track_info['acousticness'] = track_features[0]['acousticness']
    track_info['valence'] = track_features[0]['valence']

    # Extract track URL
    track_info['track_url'] = track['external_urls']['spotify']

    # Check if all fields are filled and preview_url is not "[No preview url found]"
    if all(track_info.values()) and track_info['preview_url'] != "No preview url found":
        # Calculate angle
        track_info['angle_deg'] = EmotionCalculator.calculate_angle(arousal=track_info['arousal'], valence=track_info['valence'])

        # Determine quadrant
        track_info['quadrant'] = EmotionCalculator.determine_quadrant(angle_deg=track_info['angle_deg'])

        # Determine emotion
        track_info['emotion'] = EmotionCalculator.determine_emotion(quadrant_name=track_info['quadrant'])

        # Append track information to the list
        track_info_list.append(track_info)

In [ ]:
track_info_list

[{'id': '0mvkwaZMP2gAy2ApQLtZRv',
  'name': "It's a Beautiful Day",
  'artists': ['Michael Bublé'],
  'images': [{'height': 640,
    'url': 'https://i.scdn.co/image/ab67616d0000b273051ae642ad4a0c1329b41d99',
    'width': 640},
   {'height': 300,
    'url': 'https://i.scdn.co/image/ab67616d00001e02051ae642ad4a0c1329b41d99',
    'width': 300},
   {'height': 64,
    'url': 'https://i.scdn.co/image/ab67616d00004851051ae642ad4a0c1329b41d99',
    'width': 64}],
  'preview_url': 'https://p.scdn.co/mp3-preview/4c922c3bfb3b95b4e49fb6f76b055edae8432240?cid=7a5ccd9e742a4adc9e73041d836087be',
  'arousal': 0.795,
  'danceability': 0.532,
  'acousticness': 0.0559,
  'valence': 0.78,
  'track_url': 'https://open.spotify.com/track/0mvkwaZMP2gAy2ApQLtZRv',
  'angle_deg': 46.49433359126654,
  'quadrant': 'QUAD1',
  'emotion': 'Happiness'},
 {'id': '5fnA9mkIfScSqHIpeDyvck',
  'name': 'Prayer in C - Robin Schulz Radio Edit',
  'artists': ['Lilly Wood and The Prick', 'Robin Schulz'],
  'images': [{'height'

In [ ]:
i = 0

for track_info in track_info_list:
      # Increment the track number
      i += 1

      # Printing track information
      print(f"\nTrack {i}:")
      print("Track ID:", track_info['id'])
      print("Track Name:", track_info['name'])

      # Printing artists
      print("Artists:", ', '.join(track_info['artists']))

      # print track url
      print("Track URL:", track_info['track_url'])

      # Printing preview URL
      print("Preview URL:", track_info['preview_url'])

      # Printing Track Features
      print("Arousal:", track_info['arousal'])
      print("Valence:", track_info['valence'])

      # Printing Track Quadrants
      print("Angle:", track_info['angle_deg'])
      print("Quadrant:", track_info['quadrant'])
      print("Emotion:", track_info['emotion'])


Track 1:
Track ID: 0mvkwaZMP2gAy2ApQLtZRv
Track Name: It's a Beautiful Day
Artists: Michael Bublé
Track URL: https://open.spotify.com/track/0mvkwaZMP2gAy2ApQLtZRv
Preview URL: https://p.scdn.co/mp3-preview/4c922c3bfb3b95b4e49fb6f76b055edae8432240?cid=7a5ccd9e742a4adc9e73041d836087be
Arousal: 0.795
Valence: 0.78
Angle: 46.49433359126654
Quadrant: QUAD1
Emotion: Happiness

Track 2:
Track ID: 5fnA9mkIfScSqHIpeDyvck
Track Name: Prayer in C - Robin Schulz Radio Edit
Artists: Lilly Wood and The Prick, Robin Schulz
Track URL: https://open.spotify.com/track/5fnA9mkIfScSqHIpeDyvck
Preview URL: https://p.scdn.co/mp3-preview/8b4ea0fb47a5c8f4db5125f1ff69daf10045cfbb?cid=7a5ccd9e742a4adc9e73041d836087be
Arousal: 0.875
Valence: 0.8
Angle: 51.34019174590991
Quadrant: QUAD1
Emotion: Happiness

Track 3:
Track ID: 24y98VFyG8tA5Grb9hYSHn
Track Name: Inna Jamaica
Artists: Mellow Mood, Richie Campbell
Track URL: https://open.spotify.com/track/24y98VFyG8tA5Grb9hYSHn
Preview URL: https://p.scdn.co/mp3-previ

In [ ]:
your_predicted_emotion = "Happiness"

In [ ]:
# Filter tracks by predicted emotion
filtered_tracks = [track for track in track_info_list if track['emotion'] == your_predicted_emotion]

# If the length of filtered_tracks is less than 5, include tracks with other emotions
if len(filtered_tracks) < 5:
  other_tracks = [track for track in track_info_list if track['emotion'] != your_predicted_emotion]
  filtered_tracks += other_tracks[:5 - len(filtered_tracks)]

# Sort tracks by their angle_deg
filtered_tracks.sort(key=lambda track: track['angle_deg'])

# Calculate the mean of angles in filtered tracks using NumPy
mean_angle = np.mean([track['angle_deg'] for track in filtered_tracks])

# Sort tracks by their angle difference from the mean angle using NumPy
sorted_indices = np.argsort(np.abs([track['angle_deg'] - mean_angle for track in filtered_tracks]))

# Get the top 10 tracks with angles closest to the mean angle
top_10_tracks = [filtered_tracks[i] for i in sorted_indices[:10]]

top_10_tracks

[{'id': '1f5cbQtDrykjarZVrShaDI',
  'name': 'Drowning (feat. Kodak Black)',
  'artists': ['A Boogie Wit da Hoodie', 'Kodak Black'],
  'images': [{'height': 640,
    'url': 'https://i.scdn.co/image/ab67616d0000b273cdba7ee22968991250725ce1',
    'width': 640},
   {'height': 300,
    'url': 'https://i.scdn.co/image/ab67616d00001e02cdba7ee22968991250725ce1',
    'width': 300},
   {'height': 64,
    'url': 'https://i.scdn.co/image/ab67616d00004851cdba7ee22968991250725ce1',
    'width': 64}],
  'preview_url': 'https://p.scdn.co/mp3-preview/f770a1a0b3dfb5bc652f89848e45ab9d09f7dc69?cid=7a5ccd9e742a4adc9e73041d836087be',
  'arousal': 0.81,
  'danceability': 0.839,
  'acousticness': 0.501,
  'valence': 0.814,
  'track_url': 'https://open.spotify.com/track/1f5cbQtDrykjarZVrShaDI',
  'angle_deg': 44.63272439266191,
  'quadrant': 'QUAD1',
  'emotion': 'Happiness'},
 {'id': '7zpm7lTY2EZn7AfFm3mGg2',
  'name': 'Lost in Music - 1995 Remaster',
  'artists': ['Sister Sledge'],
  'images': [{'height': 64

# DF Visualization

In [ ]:
import pandas as pd

df = pd.DataFrame(top_10_tracks)

In [ ]:
df.drop(columns=['images'], inplace=True)

In [ ]:
df

,id,name,artists,preview_url,arousal,danceability,acousticness,valence,track_url,angle_deg,quadrant,emotion
0,1f5cbQtDrykjarZVrShaDI,Drowning (feat. Kodak Black),"[A Boogie Wit da Hoodie, Kodak Black]",https://p.scdn.co/mp3-preview/f770a1a0b3dfb5bc...,0.810,0.839,0.50100,0.814,https://open.spotify.com/track/1f5cbQtDrykjarZ...,44.632724,QUAD1,Happiness
1,7zpm7lTY2EZn7AfFm3mGg2,Lost in Music - 1995 Remaster,[Sister Sledge],https://p.scdn.co/mp3-preview/57d447011ac59dc6...,0.841,0.758,0.05440,0.849,https://open.spotify.com/track/7zpm7lTY2EZn7Af...,44.335731,QUAD1,Happiness
2,49SgioQeGOwNYJJYT4fqmz,"The Other Boys (feat. Kylie Minogue, Jake Shea...","[NERVO, Kylie Minogue, Jake Shears, Nile Rodgers]",https://p.scdn.co/mp3-preview/d6bdf22155adf5e0...,0.844,0.654,0.00100,0.843,https://open.spotify.com/track/49SgioQeGOwNYJJ...,45.083400,QUAD1,Happiness
3,46oKpr00CmuPJGvPZ3y9uk,It's A Pity,[Tanya Stephens],https://p.scdn.co/mp3-preview/bbb5a6736a06df66...,0.811,0.660,0.00788,0.821,https://open.spotify.com/track/46oKpr00CmuPJGv...,44.093497,QUAD1,Happiness
4,24y98VFyG8tA5Grb9hYSHn,Inna Jamaica,"[Mellow Mood, Richie Campbell]",https://p.scdn.co/mp3-preview/1ad48d9aeecdb3fa...,0.795,0.817,0.31700,0.787,https://open.spotify.com/track/24y98VFyG8tA5Gr...,45.787521,QUAD1,Happiness
5,5MuNxNox3zTanAFIO5KcTl,He's the Greatest Dancer - 1995 Remaster,[Sister Sledge],https://p.scdn.co/mp3-preview/1948926ace135bb5...,0.815,0.700,0.00115,0.837,https://open.spotify.com/track/5MuNxNox3zTanAF...,43.067440,QUAD1,Happiness
6,0mvkwaZMP2gAy2ApQLtZRv,It's a Beautiful Day,[Michael Bublé],https://p.scdn.co/mp3-preview/4c922c3bfb3b95b4...,0.795,0.532,0.05590,0.780,https://open.spotify.com/track/0mvkwaZMP2gAy2A...,46.494334,QUAD1,Happiness
7,6rrTr2HEAzlpC4KWZxF3S1,Woman Like Me (feat. Nicki Minaj),"[Little Mix, Nicki Minaj]",https://p.scdn.co/mp3-preview/14aaf400d0bc7d21...,0.849,0.757,0.17300,0.826,https://open.spotify.com/track/6rrTr2HEAzlpC4K...,46.951546,QUAD1,Happiness
8,0LqPXq2pKXnydc96PvTK3v,"Everybody Dance - 12"" Mix",[CHIC],https://p.scdn.co/mp3-preview/0a0f837011c82c6e...,0.816,0.764,0.03780,0.850,https://open.spotify.com/track/0LqPXq2pKXnydc9...,42.077527,QUAD1,Happiness
9,0aAcuVJCmIkgVKzKHGfp1K,Hustler,[Ky-Mani Marley],https://p.scdn.co/mp3-preview/6b7d2e9ad8e76776...,0.762,0.788,0.05150,0.792,https://open.spotify.com/track/0aAcuVJCmIkgVKz...,41.900368,QUAD1,Happiness
